# Llama-factory 


**README.md**
https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md

**data/README.md**
https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README_zh.md

**examples/README.md**
https://github.com/hiyouga/LLaMA-Factory/blob/main/examples/README_zh.md


**官方notebook**

[PAI-DSW - LLaMA Factory：微调LLaMA3模型实现角色扮演](https://gallery.pai-ml.com/#/preview/deepLearning/nlp/llama_factory)

[Colab - 使用 LLaMA Factory 微调 Llama-3 中文对话模型](https://colab.research.google.com/drive/1d5KQtbemerlSDSxZIfAaWXhKr30QypiK?usp=sharing)


**作者推荐教程**

[知乎 - LLaMA-Factory QuickStart](https://zhuanlan.zhihu.com/p/695287607)




[LLM基础资料整理：推理所需显存与速度](https://techdiylife.github.io/blog/blog.html?category1=c01&blogid=0058) ⬅️ 可以结合 REAMME.md 



[Qwen2 doc 提供的使用 Llama-factory 的教程(内附量化的教程)](https://qwen.readthedocs.io/en/latest/training/SFT/llama_factory.html)




[只需 30 分钟，微调 Qwen2-7B，搭建专属 AI 客服解决方案](https://mp.weixin.qq.com/s/Pb-l4vON8PvgXwRwgOt2FQ)

[LLM基础资料整理：推理所需显存与速度](https://techdiylife.github.io/blog/blog.html?category1=c01&blogid=0058)

微调, 1024 len_cutoff
<img src="https://cdn.sa.net/2024/06/26/Df73VxIl4ZmGwei.png" style="width:500">

<p><span style="color:red; font-size:32px;"> llamafactory-cli 关键参数 --help</span></p>

关键动作参数：

| 动作参数 | 参数说明 |
| --- | --- |
| version | 显示版本信息 |
| train | 命令行版本训练 |
| chat | 命令行版本推理chat |
| export | 模型合并和导出 |
| api | 启动API server，供接口调用 |
| eval | 使用mmlu等标准数据集做评测 |
| webchat | 前端版本纯推理的chat页面 |
| webui | 启动LlamaBoard前端页面，包含可视化训练，预测，chat，模型合并多个子页面 |


另外两个关键参数解释如下，后续的基本所有环节都会继续使用这两个参数

| 参数名称 | 参数说明 |
| --- | --- |
| model_name_or_path | 参数的名称（huggingface或者modelscope上的标准定义，如"meta-llama/Meta-Llama-3-8B-Instruct"）， 或者是本地下载的绝对路径，如/media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct |
| template | 模型问答时所使用的prompt模板，不同模型不同，请参考 https://github.com/hiyouga/LLaMA-Factory?tab=readme-ov-file#supported-models 获取不同模型的模板定义，否则会回答结果会很奇怪或导致重复生成等现象的出现。chat 版本的模型基本都需要指定，比如Meta-Llama-3-8B-Instruct的template 就是 llama3 |


In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.__version__

In [ ]:
! git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
! cd LLaMA-Factory

!conda create -n llama_factory python=3.10
!conda activate llama_factory

! pip install -e ".[torch,metrics, deepspeed, bitsandbytes, vllm, gptq, awq, aqlm, qwen]"

# extra: torch, torch_npu, metrics, deepspeed, bitsandbytes, vllm, galore, badam, gptq, awq, aqlm, qwen, modelscope, quality

# ! pip install flash-attn --no-build-isolation # CUDA > 11.6

# show llamafactory version
! llamafactory-cli version

# export GRADIO_SHARE = 1 
! CUDA_VISIBLE_DEVICES=0 llamafactory-cli webui  # 目前webui版本只支持单机单卡和单机多卡，如果是多机多卡请使用命令行版本

! CUDA_VISIBLE_DEVICES=0 GRADIO_SHARE=1 GRADIO_SERVER_PORT=7860 llamafactory-cli webui # 如果要开启 gradio的share功能，或者修改端口号

## 下载模型

If you have trouble with downloading models and datasets from Hugging Face, you can use ModelScope.

```bash

source /etc/network_turbo # 启动autodl代理(非必需）
export USE_MODELSCOPE_HUB=1 # `set USE_MODELSCOPE_HUB=1` for Windows
```



In [ ]:
from modelscope import snapshot_download

# 魔塔地址：https://modelscope.cn/home, 在这上面找到模型路径，修改即可
#model_path="ZhipuAI/glm-4-9b"
#model_path = "01ai/Yi-6B"
#model_path = "qwen/Qwen2-7B"

model_path = "LLM-Research/Meta-Llama-3-8B"

cache_path="/mnt/workspace"

snapshot_download(model_path, cache_dir=cache_path)

print("done")

[如何快速下载huggingface模型——全方法总结](https://zhuanlan.zhihu.com/p/663712983)

`huggingface-cli`

```bash
pip install -U huggingface_hub
export HF_ENDPOINT=https://hf-mirror.com # $env:HF_ENDPOINT = "https://hf-mirror.com" for Windows 
```

下载模型
```bash
huggingface-cli download --resume-download gpt2 --local-dir gpt2

huggingface-cli download --token hf_*** --resume-download meta-llama/Llama-3-7b-hf --local-dir Llama-2-7b-hf
```


下载数据集
```bash
huggingface-cli download --repo-type dataset --resume-download wikitext --local-dir wikitext
```
可以添加 --local-dir-use-symlinks False 参数禁用文件软链接，这样下载路径下所见即所得，详细解释请见上面提到的教程。

```bash
wget https://hf-mirror.com/hfd/hfd.sh
chmod a+x hfd.sh
export HF_ENDPOINT=https://hf-mirror.com # $env:HF_ENDPOINT = "https://hf-mirror.com" for Windows 
```

下载模型
```bash
./hfd.sh meta-llama/Llama-3-7b --hf_username YOUR_HF_USERNAME --hf_token hf_*** --tool aria2c -x 4
```

下载数据集
```bash
./hfd.sh wikitext --dataset --tool aria2c -x 4
```

https://github.com/huggingface/hf_transfer



In [ ]:
# huggingface地址：https://huggingface.co/

import os 
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com" # 注意os.environ得在import huggingface库相关语句之前执行。

import huggingface_hub
huggingface_hub.login("HF_TOKEN") # token 从 https://huggingface.co/settings/tokens 获取


from huggingface_hub import snapshot_download
model_path = "meta-llama/Llama-3-7b"
local_dir = "/mnt/workspace"

snapshot_download(
    repo_id=model_path,
    local_dir=local_dir,
    #proxies={"https": "http://localhost:7890"},
    max_workers=8,
    local_dir_use_symlinks=False
)

print("done")

In [ ]:
# transformers 原始推理

import transformers
import torch

# 切换为你下载的模型文件目录, 这里的demo是Llama-3-8B-Instruct
# 如果是其他模型，比如qwen，chatglm，请使用其对应的官方demo
model_id = "/media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

## 数据库


然后在 dataset_info.json 中注册新数据集哦

**data/README.md**
https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README_zh.md

In [ ]:
# identity.json 
import json

%cd /content/LLaMA-Factory/

NAME = "AI_NAME"
AUTHOR = ""

identity_json_path = 'data/identity.json'
identity_json_output = 'data/identity_new.json'

with open(identity_json_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

for sample in dataset:
    sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open(identity_json_output, "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=2, ensure_ascii=False)

```bash
sed -i 's/{{name}}/PonyBot/g'  data/identity.json 
sed -i 's/{{author}}/LLaMA Factory/g'  data/identity.json
```

## 训练参数

`llamafactory-cli train -h`

| 参数名称 | 参数说明 |
| --- | --- |
| stage | 当前训练的阶段，枚举值，有"sft","pt","rw","ppo"等，代表了训练的不同阶段，这里我们是有监督指令微调，所以是sft |
| do_train | 是否是训练模式 |
| dataset | 使用的数据集列表，所有字段都需要按上文在data_info.json里注册，多个数据集用","分隔 |
| dataset_dir | 数据集所在目录，这里是 data，也就是项目自带的data目录 |
| finetuning_type | 微调训练的类型，枚举值，有"lora","full","freeze"等，这里使用lora |
| output_dir | 训练结果保存的位置 |
| cutoff_len | 训练数据集的长度截断 |
| per_device_train_batch_size | 每个设备上的batch size，最小是1，如果GPU 显存够大，可以适当增加 |
| fp16 | 使用半精度混合精度训练 |
| max_samples | 每个数据集采样多少数据 |
| val_size | 随机从数据集中抽取多少比例的数据作为验证集 |

**注意**：精度相关的参数还有bf16 和pure_bf16，但是要注意有的老显卡，比如V100就无法支持bf16，会导致程序报错或者其他错误


`使用 W&B 面板`

若要使用 Weights & Biases 记录实验数据，请在 yaml 文件中添加下面的参数

```bash
report_to: wandb
run_name: test_run # 可选
```
在启动训练任务时，将 WANDB_API_KEY 设置为密钥来登录 W&B 账户。

### LoRA 微调

（增量）预训练

```bash
llamafactory-cli train examples/train_lora/llama3_lora_pretrain.yaml
```

指令监督微调

```bash
llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml
```

多模态指令监督微调
```bash
llamafactory-cli train examples/train_lora/llava1_5_lora_sft.yaml
```

奖励模型训练
```bash
llamafactory-cli train examples/train_lora/llama3_lora_reward.yaml
```

PPO 训练
```bash
llamafactory-cli train examples/train_lora/llama3_lora_ppo.yaml
```

DPO/ORPO/SimPO 训练
```bash
llamafactory-cli train examples/train_lora/llama3_lora_dpo.yaml
```

KTO 训练
```bash
llamafactory-cli train examples/train_lora/llama3_lora_kto.yaml
```

预处理数据集: 使用 tokenized_path 以加载预处理后的数据集
```bash
llamafactory-cli train examples/train_lora/llama3_preprocess.yaml
```


### QLoRA 微调 

基于 4/8 比特 Bitsandbytes 量化进行指令监督微调（推荐）
```bash
llamafactory-cli train examples/train_qlora/llama3_lora_sft_bitsandbytes.yaml
```


基于 4/8 比特 GPTQ 量化进行指令监督微调
```bash
llamafactory-cli train examples/train_qlora/llama3_lora_sft_gptq.yaml
```

基于 4 比特 AWQ 量化进行指令监督微调
```bash 
llamafactory-cli train examples/train_qlora/llama3_lora_sft_awq.yaml
```

基于 2 比特 AQLM 量化进行指令监督微调
```
llamafactory-cli train examples/train_qlora/llama3_lora_sft_aqlm.yaml
```

### 全参数微调


在单机上进行指令监督微调
```bash
FORCE_TORCHRUN=1 llamafactory-cli train examples/train_full/llama3_full_sft_ds3.yaml
```

在单机上进行全参数指令监督微调
```bash
FORCE_TORCHRUN=1 NNODES=2 RANK=0 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_full/llama3_full_sft_ds3.yaml
FORCE_TORCHRUN=1 NNODES=2 RANK=1 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_full/llama3_full_sft_ds3.yaml
```

批量预测并计算 BLEU 和 ROUGE 分数
```bash
llamafactory-cli train examples/train_full/llama3_full_predict.yaml
```


In [ ]:
import json

args = dict(
    stage="sft",                        # 进行指令监督微调
    do_train=True,
    model_name_or_path="unsloth/Qwen2-7B-Instruct-bnb-4bit", # 使用 4 bit量化版 Qwen2-7B-Instruct 模型
    dataset="identity,bajigo",      # 使用 bajigo 和自我认知数据集
    template="qwen",                     # 使用 qwen2 提示词模板
    finetuning_type="lora",                   # 使用 LoRA 适配器来节省显存
    lora_target="all",                     # 添加 LoRA 适配器至全部线性层
    output_dir="qwen2_lora",                  # 保存 LoRA 适配器的路径
    per_device_train_batch_size=2,               # 批处理大小
    gradient_accumulation_steps=4,               # 梯度累积步数
    lr_scheduler_type="cosine",                 # 使用余弦学习率退火算法
    logging_steps=10,                      # 每 10 步输出一个记录
    warmup_ratio=0.1,                      # 使用预热学习率
    save_steps=1000,                      # 每 1000 步保存一个检查点
    learning_rate=5e-5,                     # 学习率大小
    num_train_epochs=3.0,                    # 训练轮数
    max_samples=300,                      # 使用每个数据集中的 300 条样本
    max_grad_norm=1.0,   
    quantization_bit=4,                     # 使用 4 比特 QLoRA （可选，4 bit量化版）
    loraplus_lr_ratio=16.0,                   # 使用 LoRA+ 算法并设置 lambda=16.0（可选，4 bit量化版）
    fp16=True                         # 使用 float16 混合精度训练（可选，4 bit量化版）
)

json.dump(args, open("bajigo.json", "w", encoding="utf-8"), indent=2)

# %cd /content/LLaMA-Factory/

# !llamafactory-cli train bajigo.json  # 开始指令监督微调


Qwen2 documentation 

```bash
DISTRIBUTED_ARGS="
    --nproc_per_node $NPROC_PER_NODE \
    --nnodes $NNODES \
    --node_rank $NODE_RANK \
    --master_addr $MASTER_ADDR \
    --master_port $MASTER_PORT
  "
```

```bash
torchrun $DISTRIBUTED_ARGS src/train.py \
    --deepspeed $DS_CONFIG_PATH \
    --stage sft \
    --do_train \
    --use_fast_tokenizer \
    --flash_attn \
    --model_name_or_path $MODEL_PATH \
    --dataset your_dataset \
    --template qwen \
    --finetuning_type lora \
    --lora_target q_proj,v_proj\
    --output_dir $OUTPUT_PATH \
    --overwrite_cache \
    --overwrite_output_dir \
    --warmup_steps 100 \
    --weight_decay 0.1 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --ddp_timeout 9000 \
    --learning_rate 5e-6 \
    --lr_scheduler_type cosine \
    --logging_steps 1 \
    --cutoff_len 4096 \
    --save_steps 1000 \
    --plot_loss \
    --num_train_epochs 3 \
    --bf16
  ```

## 动态合并LoRA的推理

使用命令行接口
```bash
llamafactory-cli chat examples/inference/llama3_lora_sft.yaml
```

使用浏览器界面
```bash
llamafactory-cli webchat examples/inference/llama3_lora_sft.yaml
```

启动 OpenAI 风格 API
```bash
llamafactory-cli api examples/inference/llama3_lora_sft.yaml
```

In [ ]:
%cd /content/LLaMA-Factory/
import sys
import os

# 获取当前工作目录
current_path = os.getcwd()

# 拼接当前工作目录和src目录的路径
src_path = os.path.join(current_path, 'src')

# 将src目录的路径添加到sys.path的开头
sys.path.insert(0, src_path)

from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

torch_gc()
args = dict(
    model_name_or_path="unsloth/Qwen2-7B-Instruct-bnb-4bit", # 使用 4 bit量化版 Qwen2-7B-Instruct 模型
    adapter_name_or_path="qwen2_lora",            # 加载之前保存的 LoRA 适配器
    template="qwen",                     # 和训练保持一致
    finetuning_type="lora",                  # 和训练保持一致
)
chat_model = ChatModel(args)

messages = []
print("使用 `clear` 清除对话历史，使用 `exit` 退出程序。")
while True:
    query = input("\n用户: ")
    if query.strip() == "exit":
        break
    if query.strip() == "clear":
        messages = []
        torch_gc()
        print("对话历史已清除")
        continue

    messages.append({"role": "user", "content": query})
    print("AI: ", end="", flush=True)

    response = ""
    for new_text in chat_model.stream_chat(messages):
        print(new_text, end="", flush=True)
        response += new_text

print()
messages.append({"role": "assistant", "content": response})

## 合并 LoRA 适配器 和 量化

合并 LoRA 适配器

注：请勿使用量化后的模型或 quantization_bit 参数来合并 LoRA 适配器。

```bash
llamafactory-cli export examples/merge_lora/llama3_lora_sft.yaml
```

使用 AutoGPTQ 量化模型
```bash
llamafactory-cli export examples/merge_lora/llama3_gptq.yaml
```


```bash
CUDA_VISIBLE_DEVICES=0 llamafactory-cli export \
    --model_name_or_path path_to_base_model \
    --adapter_name_or_path path_to_adapter \
    --template qwen \
    --finetuning_type lora \
    --export_dir path_to_export \
    --export_size 2 \
    --export_legacy_format False

```

In [ ]:
# AWQ 量化: https://qwen.readthedocs.io/en/latest/quantization/awq.html

In [ ]:
# GPTQ 量化: https://qwen.readthedocs.io/en/latest/quantization/gptq.html 

In [ ]:
# GGUF 量化 : https://qwen.readthedocs.io/en/latest/quantization/gguf.html

## 评估

在 MMLU/CMMLU/C-Eval 上评估

```bash
llamafactory-cli eval examples/train_lora/llama3_lora_eval.yaml
```

批量预测并计算 BLEU 和 ROUGE 分数
```bash
llamafactory-cli train examples/train_lora/llama3_lora_predict.yaml
```

当然上文中的人工交互测试，会偏感性，那有没有办法批量地预测一批数据，然后使用自动化的bleu和 rouge等常用的文本生成指标来做评估。指标计算会使用如下3个库，请先做一下pip安装

```bash
pip install jieba
pip install rouge-chinese
pip install nltk
```

```bash
CUDA_VISIBLE_DEVICES=0 llamafactory-cli train \
    --stage sft \
    --do_predict \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft  \
    --dataset alpaca_gpt4_zh,identity,adgen_local \
    --dataset_dir ./data \
    --template llama3 \
    --finetuning_type lora \
    --output_dir ./saves/LLaMA3-8B/lora/predict \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_eval_batch_size 1 \
    --max_samples 20 \
    --predict_with_generate
```


| 指标 | 含义 |
| --- | --- |
| BLEU-4 | BLEU（Bilingual Evaluation Understudy）是一种常用的用于评估机器翻译质量的指标。BLEU-4 表示四元语法 BLEU 分数，它衡量模型生成文本与参考文本之间的 n-gram 匹配程度，其中 n=4。值越高表示生成的文本与参考文本越相似，最大值为 100。|
| predict_rouge-1 和 predict_rouge-2 | ROUGE（Recall-Oriented Understudy for Gisting Evaluation）是一种用于评估自动摘要和文本生成模型性能的指标。ROUGE-1 表示一元 ROUGE 分数，ROUGE-2 表示二元 ROUGE 分数，分别衡量模型生成文本与参考文本之间的单个词和双词序列的匹配程度。值越高表示生成的文本与参考文本越相似，最大值为 100。|
| predict_rouge-l | ROUGE-L 衡量模型生成文本与参考文本之间最长公共子序列（Longest Common Subsequence）的匹配程度。值越高表示生成的文本与参考文本越相似，最大值为 100。|
| predict_runtime | 预测运行时间，表示模型生成一批样本所花费的总时间。单位通常为秒。|
| predict_samples_per_second | 每秒生成的样本数量，表示模型每秒钟能够生成的样本数量。通常用于评估模型的推理速度。|
| predict_steps_per_second | 每秒执行的步骤数量，表示模型每秒钟能够执行的步骤数量。对于生成模型，一般指的是每秒钟执行生成操作的次数。|

## 部署(vllm, xinference)


训练好后，可能部分同学会想将模型的能力形成一个可访问的网络接口，通过API 来调用，接入到langchian或者其他下游业务中，项目也自带了这部分能力。

API 实现的标准是参考了OpenAI的相关接口协议，基于uvicorn服务框架进行开发， 使用如下的方式启动

Deploy with OpenAI-style API and vLLM

```bash
API_PORT=8000 llamafactory-cli api examples/inference/llama3_vllm.yaml
```


```bash
CUDA_VISIBLE_DEVICES=0 API_PORT=8000 llamafactory-cli api \
    --model_name_or_path /media/codingma/LLM/llama3/Meta-Llama-3-8B-Instruct \
    --adapter_name_or_path ./saves/LLaMA3-8B/lora/sft \
    --template llama3 \
    --finetuning_type lora
```

项目也支持了基于vllm 的推理后端，但是这里由于一些限制，需要提前将LoRA 模型进行merge，使用merge后的完整版模型目录或者训练前的模型原始目录都可

```bash
CUDA_VISIBLE_DEVICES=0 API_PORT=8000 llamafactory-cli api \
    --model_name_or_path megred-model-path \
    --template llama3 \
    --infer_backend vllm \
    --vllm_enforce_eager
```

In [ ]:
# https://qwen.readthedocs.io/en/latest/deployment/vllm.html

In [ ]:
import os
from openai import OpenAI
from transformers.utils.versions import require_version

require_version("openai>=1.5.0", "To fix: pip install openai>=1.5.0")

if __name__ == '__main__':
    # change to your custom port
    port = 8000
    client = OpenAI(
        api_key="0",
        base_url="http://localhost:{}/v1".format(os.environ.get("API_PORT", 8000)),
    )
    messages = []
    messages.append({"role": "user", "content": "hello, where is USA"})
    result = client.chat.completions.create(messages=messages, model="test")
    print(result.choices[0].message)

## Ollama  + GGUF 

create a 'Modelfile'

```text
FROM qwen2-7b-instruct-q5_0.gguf

# set the temperature to 1 [higher is more creative, lower is more coherent]
PARAMETER temperature 0.7
PARAMETER top_p 0.8
PARAMETER repeat_penalty 1.05
PARAMETER top_k 20

TEMPLATE """{{ if and .First .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
{{ .Response }}"""

# set the system message
SYSTEM """
You are a helpful assistant.
"""
```

```bash
ollama create qwen2_7b -f Modelfile
ollama run qwen2_7b
```